# Experiments with GP pushing and then simulated annealing for finalization

In [1]:
%load_ext autoreload
%autoreload 2
from DLC_for_WBFM.utils.projects.finished_project_data import finished_project_data
import numpy as np
import matplotlib.pyplot as plt
import pickle
import napari, zarr

In [190]:
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"
dat = finished_project_data.load_final_project_data_from_config(fname)

In [3]:
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\2-training_data\raw\match_dat.pickle"
with open(fname, 'rb') as f:
    matches = pickle.load(f)
    
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\2-training_data\raw\frame_dat.pickle"
with open(fname, 'rb') as f:
    frames = pickle.load(f)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [192]:
cfg = dat.unpack_config_file(fname)[0]

In [194]:
cfg.config.

ordereddict([('task_name', 'test'), ('experimenter', 'Charlie'), ('project_dir', 'C:/dlc_stacks/Charlie-worm3-new-seg'), ('project_path', None), ('red_bigtiff_fname', 'Y:/shared_projects/wbfm/dat/ZIM2051/2021-03-04_16-07-57_worm3_ZIM2051/2021-03-04_16-07-57_worm3_ZIM2051-channel-0-pco_camera1/2021-03-04_16-07-57_worm3_ZIM2051-channel-0-pco_camera1bigtiff.btf'), ('green_bigtiff_fname', 'Y:/shared_projects/wbfm/dat/ZIM2051/2021-03-04_16-07-57_worm3_ZIM2051/2021-03-04_16-07-57_worm3_ZIM2051-channel-1-pco_camera2/2021-03-04_16-07-57_worm3_ZIM2051-channel-1-pco_camera2bigtiff.btf'), ('preprocessing_config', 'preprocessing_config.yaml'), ('preprocessed_red', 'D:/2021-03-04_16-07-57_worm3_ZIM2051/2021-03-04_16-07-57_worm3_ZIM2051-channel-0-pco_camera1/2021-03-04_16-07-57_worm3_ZIM2051-channel-0-pco_camera1bigtiff.zarr'), ('preprocessed_green', 'D:/2021-03-04_16-07-57_worm3_ZIM2051/2021-03-04_16-07-57_worm3_ZIM2051-channel-1-pco_camera2/2021-03-04_16-07-57_worm3_ZIM2051-channel-1-pco_camera2bi

# Visualize the pushed neurons on the next frame

In [4]:
pair = (0, 1)
print(matches[pair])
print(frames[pair[0]], frames[pair[1]])

FramePair with 134 matches 

ReferenceFrame:
Frame index: 0 
Number of neurons: 148 
Number of keypoints: 148 
ReferenceFrame:
Frame index: 1 
Number of neurons: 151 
Number of keypoints: 151 



In [5]:
# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

In [6]:
# Get gp pushed locations

gp_zxy = matches[pair].gp_pushed_locations
# Replace z coordinate, which is somehow crazy
gp_zxy[:, 0] = frames[pair[0]].neuron_locs[:, 0]
gp_zxy = gp_zxy[:, [0,2,1]]

affine_zxy = matches[pair].affine_pushed_locations
# Replace z coordinate, which is somehow crazy
# affine_zxy[:, 0] = frames[pair[0]].neuron_locs[:, 0]
affine_zxy = affine_zxy[:, [0,2,1]]

In [7]:
# Split by "has match"
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}

# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_xyz_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_xyz_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

gp_xyz_matched = [row for i, row in enumerate(gp_zxy) if i in has_match_ind]
gp_xyz_unmatched = [row for i, row in enumerate(gp_zxy) if i not in has_match_ind]

affine_xyz_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_xyz_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [8]:
# # Plot using napari
# v = napari.view_image(dat.red_data[pair[1]], ndisplay=3)
# # v.add_points(gp_xyz_matched, size=3, face_color='green', n_dimensional=True)
# v.add_points(gp_xyz_unmatched, size=3, face_color='red', n_dimensional=True)
# # v.add_points(affine_xyz_matched, size=3, face_color='green', symbol='x', n_dimensional=True)
# v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
# v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True)
# v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True)

In [9]:

# Get gradient images (not in z)
# vol = dat.red_data[pair[1], ...]
# vol_filtered = gaussian(vol, sigma=1)
# thresh = 1.5*np.mean(vol_filtered)
# mask = vol_filtered > thresh
# vol_gradient_x = sobel_h(vol_filtered, mask)
# vol_gradient_y = sobel_v(vol_filtered, mask)

# Actually do it (no annealing)

In [10]:
# from skimage.filters import gaussian, sobel_h, sobel_v
from scipy.spatial import KDTree

In [11]:
pair = (1, 2)

In [12]:
# Get initial affine locations
affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Get image
vol = dat.red_data[pair[1], ...]
# vol_filtered = gaussian(vol, sigma=1)

# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

In [13]:
# Only check the ones with no match
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}
has_match_ind = list(f0_to_f1_matches.keys())
no_match_ind = list(range(frames[pair[0]].num_neurons()))
[no_match_ind.remove(i) for i in has_match_ind]

# Set up kdtree to check nearest neighbor
tree = KDTree(target_zxy)

# Parameters
brightness_thresh = 25
distance_thresh = 5.0

In [14]:
new_f1_zxy = []
new_matches = []
next_f1_ind = frames[pair[1]].num_neurons() + 1

for ind_f0 in no_match_ind:
    pushed_zxy = affine_zxy[ind_f0, :]
    z, x, y = int(pushed_zxy[0]), int(pushed_zxy[1]), int(pushed_zxy[2])
    brightness = vol[z, x, y]
    
    nn_dist, _ = tree.query(pushed_zxy, k=1)
    
    to_keep = (brightness > brightness_thresh) and (nn_dist > distance_thresh)
    print(to_keep, brightness, nn_dist)
    if to_keep:
        new_matches.append([ind_f0, next_f1_ind, 1.0])
        next_f1_ind += 1
        new_f1_zxy.append(pushed_zxy)

print(f"Added {len(new_matches)} neurons")

False 0 278.634677184079
False 22 6.514810434866421
True 40 8.798306660844792
False 31 3.65074508888712
False 34 4.273267236144742
True 36 9.445009640142503
True 27 12.186874485455414
True 35 7.80057086110525
Added 4 neurons


In [15]:
# # Plot using napari
# v = napari.view_image(dat.red_data[pair[1]], ndisplay=3)
# v.add_points(new_f1_zxy, size=3, face_color='green', symbol='x', n_dimensional=True)
# v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
# v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True)
# v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True)

In [16]:
f0, f1 = frames[pair[0]], frames[pair[1]]

In [17]:
len(f0.keypoint_locs)

151

# Use piecewise affine with only matched neurons

In [18]:
from skimage.transform import PiecewiseAffineTransform, warp

In [19]:
pair = (0, 1)
print(matches[pair])
print(frames[pair[0]], frames[pair[1]])

FramePair with 134 matches 

ReferenceFrame:
Frame index: 0 
Number of neurons: 148 
Number of keypoints: 148 
ReferenceFrame:
Frame index: 1 
Number of neurons: 151 
Number of keypoints: 151 



In [20]:
m = matches[pair]
f0, f1 = frames[pair[0]], frames[pair[1]]

In [21]:
f0_to_f1 = m.get_f0_to_f1_dict()

In [22]:
src_matched = np.array([f0.keypoint_locs[k,[2,1]] for k in f0_to_f1.keys()])
src_all_xy = np.array(f0.keypoint_locs[:,[2,1]])
src_z = np.array(f0.keypoint_locs[:,[0]])

dst_matched = np.array([f1.keypoint_locs[v,[2,1]] for v in f0_to_f1.values()])
dst_all_xy = np.array(f0.keypoint_locs[:,[2,1]])
dst_z = np.array(f0.keypoint_locs[:,[0]])

In [23]:

tform = PiecewiseAffineTransform()
tform.estimate(src_matched, dst_matched)

True

In [24]:
transformed = tform.inverse(dst_all_xy)
# src_transformed = tform.inverse(src)

transformed = np.where(transformed==-1, np.nan, transformed)

transformed = np.hstack([dst_z, transformed]) #[:, [0,2,1]]

In [25]:
# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Split by "has match"
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}

# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_xyz_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_xyz_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

affine_xyz_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_xyz_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [26]:
# # Plot using napari
# v = napari.view_image(dat.red_data[pair[0]], ndisplay=3, name='f0_red')
# v.add_image(dat.red_data[pair[1]], name='f1_red')
# v.add_points(transformed, size=3, face_color='red', n_dimensional=True, name='piecewise_affine_transformed')
# v.add_points(affine_xyz_matched, size=3, face_color='green', symbol='x', n_dimensional=True, name='f0_affine_matched')
# # v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
# v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True, name='f1_matched')
# v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True, name='f1_unmatched')

In [27]:

# tform.estimate(dst, src)
# src_transformed = tform.inverse(src)
# # src_transformed = tform.inverse(f0.keypoint_locs[:,[2,1]])

# src_transformed[:3], dst[:3], src[:3]

# Piecewise affine, but add additional ORB features

In [28]:
from skimage.transform import PiecewiseAffineTransform, warp

In [29]:
pair = (0, 1)
pair_obj = matches[pair]
print(pair_obj)
f0, f1 = frames[pair[0]], frames[pair[1]]
print(f0, f1)

FramePair with 134 matches 

ReferenceFrame:
Frame index: 0 
Number of neurons: 148 
Number of keypoints: 148 
ReferenceFrame:
Frame index: 1 
Number of neurons: 151 
Number of keypoints: 151 



In [30]:
# Update Frame classes, if old style
pair_obj.frame0, pair_obj.frame1 = f0, f1

In [31]:
err

NameError: name 'err' is not defined

In [41]:
f0

ReferenceFrame with 148 neurons 

In [ ]:
video_dat = dat.red_data[pair[0],...]

# f0.detect_non_neuron_keypoints(video_dat, append_to_existing_keypoints=True)
f0.encode_all_keypoints(video_dat, z_depth=0)
# f0.build_nontrivial_keypoint_to_neuron_mapping(neuron_feature_radius=3)

In [ ]:
from DLC_for_WBFM.utils.feature_detection.utils_features import convert_to_grayscale
import cv2

loc = f0.neuron_locs[0]

z, x, y = loc
kp_2d = cv2.KeyPoint(x, y, 31.0)
base_2d_encoder = cv2.xfeatures2d.VGG_create()
im_3d_gray = [convert_to_grayscale(xy).astype('uint8') for xy in video_dat]
im_2d = im_3d_gray[int(5)]
_, this_ds = base_2d_encoder.compute(im_2d, [kp_2d])

In [ ]:
f0.all_features.shape

In [ ]:
m = matches[pair]
f0, f1 = frames[pair[0]], frames[pair[1]]

In [ ]:
f0_to_f1 = m.get_f0_to_f1_dict()

In [ ]:
src_matched = np.array([f0.neuron_locs[k,[2,1]] for k in f0_to_f1.keys()])
src_all_xy = np.array(f0.neuron_locs[:,[2,1]])
src_z = np.array(f0.neuron_locs[:,[0]])

dst_matched = np.array([f1.neuron_locs[v,[2,1]] for v in f0_to_f1.values()])
dst_all_xy = np.array(f0.neuron_locs[:,[2,1]])
dst_z = np.array(f0.neuron_locs[:,[0]])

In [ ]:
tform = PiecewiseAffineTransform()
tform.estimate(src_matched, dst_matched)

In [ ]:
transformed = tform.inverse(dst_all_xy)
# src_transformed = tform.inverse(src)

transformed = np.where(transformed==-1, np.nan, transformed)

transformed = np.hstack([dst_z, transformed]) #[:, [0,2,1]]

In [ ]:
# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Split by "has match"
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}

# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_xyz_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_xyz_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

affine_xyz_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_xyz_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [ ]:
# Plot using napari
v = napari.view_image(dat.red_data[pair[0]], ndisplay=3, name='f0_red')
v.add_image(dat.red_data[pair[1]], name='f1_red')
v.add_points(transformed, size=3, face_color='red', n_dimensional=True, name='piecewise_affine_transformed')
v.add_points(affine_xyz_matched, size=3, face_color='green', symbol='x', n_dimensional=True, name='f0_affine_matched')
# v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True, name='f1_matched')
v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True, name='f1_unmatched')

# Basic encoding tests: do matches change based on VGG z depth?

In [50]:
from DLC_for_WBFM.utils.feature_detection.class_frame_pair import calc_FramePair_like

In [47]:
pair = (0, 1)
pair_obj = matches[pair]
print(pair_obj)
f0, f1 = frames[pair[0]], frames[pair[1]]
# print(f0, f1)

FramePair with 134/148 matches 



In [132]:
# Fix old style and add metadata
pair_obj.frame0, pair_obj.frame1 = f0, f1

pair_obj.add_affine_to_candidates=True
pair_obj.add_gp_to_candidates=True

pair_obj.min_confidence = 0.3

In [133]:
f0_video_dat = dat.red_data[pair[0],...]
f1_video_dat = dat.red_data[pair[1],...]

# Redo encoding with no z depth
f0.rebuild_keypoints()
f0.encode_all_keypoints(f0_video_dat, z_depth=5)
f1.rebuild_keypoints()
f1.encode_all_keypoints(f1_video_dat, z_depth=5);
# Note: do not need to change keypoint-neuron matching

Overwriting existing keypoints...


  0%|          | 0/148 [00:00<?, ?it/s]

Overwriting existing keypoints...


  0%|          | 0/151 [00:00<?, ?it/s]

In [134]:
# Redo matching (new pair object)
new_pair5 = calc_FramePair_like(pair_obj, f0, f1)

# Compare
print(new_pair5)

FramePair with 120/148 matches 



C:\Users\charles.fieseler\Anaconda3\envs\segmentation\lib\site-packages\sklearn\gaussian_process\kernels.py:409: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__sigma_0 is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
C:\Users\charles.fieseler\Anaconda3\envs\segmentation\lib\site-packages\sklearn\gaussian_process\kernels.py:409: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  ConvergenceWarning)


In [135]:
f0_video_dat = dat.red_data[pair[0],...]
f1_video_dat = dat.red_data[pair[1],...]

# Redo encoding with no z depth
f0.rebuild_keypoints()
f0.encode_all_keypoints(f0_video_dat, z_depth=0)
f1.rebuild_keypoints()
f1.encode_all_keypoints(f1_video_dat, z_depth=0);

Overwriting existing keypoints...


  0%|          | 0/148 [00:00<?, ?it/s]

Overwriting existing keypoints...


  0%|          | 0/151 [00:00<?, ?it/s]

In [136]:
# Redo matching (new pair object)
new_pair0 = calc_FramePair_like(pair_obj, f0, f1)

# Compare
print(new_pair0)

C:\Users\charles.fieseler\Anaconda3\envs\segmentation\lib\site-packages\sklearn\gaussian_process\kernels.py:409: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__sigma_0 is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  ConvergenceWarning)


FramePair with 127/148 matches 



In [137]:
f0_map_0 = new_pair0.get_f0_to_f1_dict()
f0_map_5 = new_pair5.get_f0_to_f1_dict()

f0_conf_0 = new_pair0.get_pair_to_conf_dict()
f0_conf_5 = new_pair5.get_pair_to_conf_dict()

In [138]:
num_mismatches = 0
all_keys = set(f0_map_0.keys())
all_keys.update(tuple(f0_map_5.keys()))
for i in all_keys:
    m0, m5 = f0_map_0.get(i,None), f0_map_5.get(i,None)
    if m0 != m5:
        print(i, ':', m0, m5)
        print('conf: ', f0_conf_0.get((i, m0), None), f0_conf_5.get((i, m5), None))
        num_mismatches += 1
print(f"Number of mismatches {num_mismatches}")

0 : 68 None
conf:  0.5802337759857685 None
7 : 130 None
conf:  0.5486832314830761 None
13 : 138 None
conf:  0.5371344019298478 None
39 : 133 83
conf:  0.8262777632713082 0.6598924939585387
53 : None 3
conf:  None 0.8526980230820976
73 : 35 None
conf:  0.980316170613662 None
89 : 104 141
conf:  0.83746353324169 0.5308542620316973
92 : 123 147
conf:  0.9653892651475615 0.8826619719723844
96 : 26 None
conf:  0.5436642479928631 None
119 : None 72
conf:  None 0.33635003150777815
134 : 145 None
conf:  0.8160640144787115 None
138 : 106 None
conf:  0.4602121839039434 None
139 : None 150
conf:  None 0.6968154628490295
140 : 31 None
conf:  0.4584115341724631 None
144 : 119 None
conf:  0.7999233563587401 None
145 : 3 None
conf:  0.5017519312086033 None
Number of mismatches 16


# Prep for manual correction

In [150]:
target_dir = '.'
pair_obj.save_matches_as_excel(target_dir)

In [151]:
import pandas as pd
fname = r"C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_for_WBFM\alternative_ideas\matches_f0_f1.xlsx"
df = pd.read_excel(fname)

In [179]:
# df

# Check methods using manual ground truth

In [ ]:
from DLC_for_WBFM.utils.performance.comparing_ground_truth import *

In [180]:
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\2-training_data\manual_tracking\matches_f0_f1.xlsx"
df_gt = pd.read_excel(fname)

In [182]:
gt_f0_to_f1 = {n0: n1 for n0, n1 in zip(df_gt['Neuron_in_f0'], df_gt['Neuron_in_f1'])}

In [188]:
tp_0 = calc_true_positive(gt_f0_to_f1, f0_map_0)
mismatch_0 = calc_mismatches(gt_f0_to_f1, f0_map_0)
missing_0 = calc_missing_matches(gt_f0_to_f1, f0_map_0)

print(f'tp: {tp_0}, mismatch: {mismatch_0}, missing: {missing_0}')

tp: 113, mismatch: 14, missing: 18


In [189]:
tp_5 = calc_true_positive(gt_f0_to_f1, f0_map_5)
mismatch_5 = calc_mismatches(gt_f0_to_f1, f0_map_5)
missing_5 = calc_missing_matches(gt_f0_to_f1, f0_map_5)

print(f'tp: {tp_5}, mismatch: {mismatch_5}, missing: {missing_5}')

tp: 118, mismatch: 2, missing: 23


# Scratch: sanity checks for the frames

In [ ]:
fname = r"C:\dlc_stacks\Test_project\2-training_data\all_tracklets.h5"
import pandas as pd

df = pd.read_hdf(fname)

In [ ]:
df

In [ ]:
df.diff()[1:]


In [ ]:
all_neurons = list(df.columns.levels[0])
for n in all_neurons:
    zd = df.diff()[1:][n]['z']
    if all(abs(zd) > 2):
        print(n, list(zd))

In [ ]:
np.quantile(np.arange(10), 0.9)

In [79]:
np.arange(5,6)

array([5])

In [155]:
# fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\4-traces\green_traces.h5"
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\2-training_data\all_tracklets.h5"
df = pd.read_hdf(fname)

In [156]:
df


bodyparts   neuron1                                       neuron3              \
coords            z           x           y likelihood          z           x   
0          8.446753  498.350649  275.957143        1.0  22.439336  314.361430   
1          8.444737  484.360526  289.947368        1.0  22.398551  300.369565   
2          6.431944  480.381944  283.933333        1.0  24.373353  306.052709   
3          8.410738  472.357047  267.971812        1.0  24.356125  287.955840   
4          8.450135  486.397574  247.900270        1.0  22.410832  296.363276   
...             ...         ...         ...        ...        ...         ...   
1493            NaN         NaN         NaN        NaN        NaN         NaN   
1494            NaN         NaN         NaN        NaN        NaN         NaN   
1495            NaN         NaN         NaN        NaN        NaN         NaN   
1496            NaN         NaN         NaN        NaN        NaN         NaN   
1497            NaN         NaN         NaN        NaN        NaN         NaN   

bodyparts                           neuron4              ... neuron14933  \
coords              y likelihood          z           x  ...           y   
0          285.966794        1.0  18.452477  238.413655  ...         NaN   
1          238.050725        1.0  18.310245  220.197691  ...         NaN   
2          235.695461        1.0  18.430311  234.339648  ...         NaN   
3          231.786325        1.0  16.334828  206.173393  ...         NaN   
4          265.988111        1.0  18.415531  212.557221  ...         NaN   
...               ...        ...        ...         ...  ...         ...   
1493              NaN        NaN        NaN         NaN  ...  194.364000   
1494              NaN        NaN        NaN         NaN  ...  218.574000   
1495              NaN        NaN        NaN         NaN  ...  248.150685   
1496              NaN        NaN        NaN         NaN  ...  261.994513   
1497              NaN        NaN        NaN         NaN  ...  276.023129   

bodyparts            neuron14942                                     \
coords    likelihood           z           x           y likelihood   
0                NaN         NaN         NaN         NaN        NaN   
1                NaN         NaN         NaN         NaN        NaN   
2                NaN         NaN         NaN         NaN        NaN   
3                NaN         NaN         NaN         NaN        NaN   
4                NaN         NaN         NaN         NaN        NaN   
...              ...         ...         ...         ...        ...   
1493             1.0   16.865350  430.815081  228.280072        1.0   
1494             1.0   18.389344  424.262295  237.949454        1.0   
1495             1.0   18.568794  388.365957  257.853901        1.0   
1496             1.0   16.970990  390.798635  265.643345        1.0   
1497             1.0   16.878354  394.506261  273.343470        1.0   

bodyparts neuron14947                                     
coords              z           x           y likelihood  
0                 NaN         NaN         NaN        NaN  
1                 NaN         NaN         NaN        NaN  
2                 NaN         NaN         NaN        NaN  
3                 NaN         NaN         NaN        NaN  
4                 NaN         NaN         NaN        NaN  
...               ...         ...         ...        ...  
1493        20.689394  430.340909  279.860606        1.0  
1494        20.585859  446.311688  281.793651        1.0  
1495        20.540382  428.086637  289.975037        1.0  
1496        20.395722  436.323529  285.927807        1.0  
1497        18.560519  446.168588  285.969741        1.0  

[1498 rows x 22200 columns]

In [164]:
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\2-training_data\raw\match_dat.pickle"
import pickle
with open(fname, 'rb') as f:
    matches = pickle.load(f)

In [165]:
matches[(0,1)].final_matches

[(0, 132, 0.19394127314555554),
 (1, 13, 0.8909203601726373),
 (2, 21, 0.6891357463350856),
 (3, 60, 0.7933977349967598),
 (4, 135, 0.6513890239764673),
 (5, 15, 0.8782441680213988),
 (6, 63, 0.5489565919109868),
 (7, 130, 0.18145020257382444),
 (9, 57, 0.5326748136395983),
 (10, 16, 0.5756298461358522),
 (11, 52, 0.9507971331859374),
 (12, 69, 0.8932474953010683),
 (13, 138, 0.17542194947816742),
 (14, 7, 0.8866151172334347),
 (15, 115, 0.5482619015814976),
 (16, 42, 0.8694483281384128),
 (17, 1, 0.8828029321421539),
 (18, 4, 0.9504420227414255),
 (19, 0, 0.6024151438447899),
 (20, 5, 0.6391916308619207),
 (21, 96, 0.8768781131841831),
 (22, 36, 0.7434910172759591),
 (23, 12, 0.5947882001962271),
 (24, 30, 0.479254314453236),
 (25, 58, 0.8779022716123891),
 (26, 6, 0.899642420290553),
 (27, 51, 0.6878049108466145),
 (28, 23, 0.9101107094656156),
 (29, 22, 0.9786965174455088),
 (30, 53, 0.5023898674371623),
 (31, 54, 0.7715761425091863),
 (32, 17, 0.912707603924168),
 (33, 33, 0.660378

In [172]:
from DLC_for_WBFM.utils.visualization.napari_from_config import napari_labels_from_frames

In [176]:
opt = napari_labels_from_frames(frames, num_frames=1)

In [178]:
# opt